In [49]:
function Request {
    <#
    In some cases you may not know exceptions ahead of time, or, it's easier to regex match on them.
    Make sure you throw any exceptions you don't handle yourself. 
    #>
    param(
        # suggested urls
        [ArgumentCompletions('https://httpbin.org/status/401', 'https://httpbin.org/status/403', 'https://httpbin.org/status/200')]
        [Parameter(Mandatory)]
        [uri]$Url
    ) 
    try {
        $response = Invoke-RestMethod $Url -ea stop
    }
    catch { 
        if ( $_.Exception.ToString() -match 'status.*(401|403|418)' ) { 
            "Bad stuff. Auth/Auth failed: $_";
            #  $_ | Write-Host -ForegroundColor blue
            return
        }
        else { 
            throw $_ # return the rest
        }
    }
    # $REponse.gettype()
    return $Response
}

'Good response: '
$resp = Request -url 'https://httpbin.org/status/200'

'Handling exceptions that you care about'
Request -url 'https://httpbin.org/status/401'
Request -url 'https://httpbin.org/status/403'
Request -url 'https://httpbin.org/status/418'

'Test that exceptions out-of-scope properly throw'
Request -url 'https://httpbin.org/status/500'

Good response: 
Handling exceptions that you care about
Bad stuff. Auth/Auth failed: Response status code does not indicate success: 401 (UNAUTHORIZED).
Bad stuff. Auth/Auth failed: Response status code does not indicate success: 403 (FORBIDDEN).
Bad stuff. Auth/Auth failed: 
    -=[ teapot ]=-

       _...._
     .'  _ _ `.
    | ."` ^ `". _,
    \_;`"---"`|//
      |       ;/
      \_     _/
        `"""`

Test that exceptions out-of-scope properly throw
Invoke-RestMethod: 
Line |
   9 |          $response = Invoke-RestMethod $Url -ea stop
     |                      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Response status code does not indicate success: 500 (INTERNAL SERVER ERROR).


### Simpler IRM Exception handling `irm -SkipHttpErrorCheck -StatusCodeVariable`

In the other examples I don't use `SkipHttpErrorCheck` because I'm using IRM to reproducibly throw multiple exception types.

For your code, try either parameters: `-StatusCodeVariable` and `-SkipHttpErrorCheck`
```ps1
Invoke-RestMethod -StatusCodeVariable 'status' -Uri $Url
Invoke-RestMethod -StatusCodeVariable 'status' -Uri $Url -SkipHttpErrorCheck
```

It simplifies having overlapping code

In [65]:
Invoke-RestMethod -StatusCodeVariable 'status' -Uri 'https://httpbin.org/status/404' -SkipHttpErrorCheck

function Request2 {
    <#
    In some cases you may not know exceptions ahead of time, or, it's easier to regex match on them.
    #>
    param(
        # suggested urls
        [ArgumentCompletions('https://httpbin.org/status/401', 'https://httpbin.org/status/403', 'https://httpbin.org/status/200')]
        [Parameter(Mandatory)]
        [uri]$Url
    ) 
    $response = Invoke-RestMethod $Url -ea stop -StatusCodeVariable 'status' -SkipHttpErrorCheck
    
    # (I could/should have used a `switch -regex` )
    switch ($Status) { 
        # this fires on both
        { $_ -match '40\d+' } { "Request Errors! ; $_" | write-warning }

        #in addition, 404 fires
        { $_ -match '404' } { 
            'bad url, missing page. Or if using a web API it can mean that you have permission to query a SQL table, but only employees in your region' | Write-Warning
        }  
        { $_ -in @('403', '401') } {
            'Auth related issues'    | Write-warning 
        }
        '403' { 'RefreshToken()' }
        '401' { 'UserLogin()' }
        '200' { return $Response } # It's good!
    }
    '-' * 40 -join '' | write-host -fore blue
}

In [66]:
'Good response: '
$resp = Request2 -url 'https://httpbin.org/status/200'

'Handling exceptions that you care about'
Request2 -url 'https://httpbin.org/status/401'
Request2 -url 'https://httpbin.org/status/403'
Request2 -url 'https://httpbin.org/status/418'

'Test that exceptions out-of-scope properly throw'
Request2 -url 'https://httpbin.org/status/500'

Good response: 
Handling exceptions that you care about
UserLogin()
----------------------------------------
RefreshToken()
----------------------------------------
----------------------------------------
Test that exceptions out-of-scope properly throw
----------------------------------------
